# Extracting the data from the .csv file

In [1]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:, -1]

# Balancing The dataset

- Step 1: we will count the number of targets that are ones as we know they are less than the zeros.
- Step 2: we will keep as many zeros as there are ones(we will delete the others).

In [2]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_be_removed = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_be_removed.append(i)

unscaled_inputs_equal_prior = np.delete(unscaled_inputs_all, indices_to_be_removed, axis = 0)
targets_equal_prior = np.delete(targets_all, indices_to_be_removed, axis = 0)

# Standardizing the inputs

In [3]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_prior)

# Shuffling the data

Since we will be batching, we must shuffle the data. It should be as randomly spread as possible so batching will work fine.


In [4]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_prior[shuffled_indices]

# Splitting the dataset into train, validation and test

In [5]:
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

# It is useful to check if we have balanced the dataset. Sometimes we balance the whole dataset, but not the train, test and validation datasets.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1799.0 3579 0.5026543727298128
218.0 447 0.48769574944071586
220.0 448 0.49107142857142855


# Saving the datasets into 3 .npz files

In [6]:
np.savez('Audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)